# Scraping HealthGrades for References

In [6]:
!pip3 install lxml
#import all tools for project
import requests
import pandas as pd
from bs4 import BeautifulSoup
import json
import numpy as np
import datetime
import matplotlib.pyplot as plt
import re
from time import sleep

You should consider upgrading via the '/usr/local/bin/python3.8 -m pip install --upgrade pip' command.


In [139]:
#example for exactly one doctor

url = 'https://www.healthgrades.com/physician/dr-craig-ehrensing-yxw7q'
def get_reviews(url):
    '''
    Takes in a url to any specific doctor and returns all of their reviews as strs in a list
    '''
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    response = requests.get(url, headers=headers)

    #use Beautiful Soup to prettify the content
    soup = BeautifulSoup(response.content)
    soup.prettify()
    
    gender = None
    #determine gender 
    headdiv = soup.findAll("div", {"class":"summary-row summary-row-top"})
    if headdiv:
        for tag in headdiv:  
            genderInfo = tag.findAll("span", {"data-qa-target":"ProviderDisplayGender"})
            genderStr = str(genderInfo)
            result = re.search('<span data-qa-target="ProviderDisplayGender"> <!-- -->(.*)<!-- --> </span>', genderStr)
            gender = result.group(1)
    else:
        headdiv = soup.findAll("div", {"class": "summary-designated-header-image-left"})
        for tag in headdiv:  
            genderInfo = tag.findAll("span", {"data-qa-target":"ProviderDisplayGender"})
            genderStr = str(genderInfo)
            result = re.search('<span data-qa-target="ProviderDisplayGender"> <!-- -->(.*)<!-- --> </span>', genderStr)
            gender = result.group(1)
            
    #retrieve all comments
    commentdivs = soup.findAll("div", {"class": "c-single-comment__comment"})
    commentdivs
    reviews = []
    for div in commentdivs:
        s = str(div)
        s = s[1:]
        start = s.find(">")+1
        end = s.find("<")
        substring = s[start:end]
        reviews.append(substring)
    return gender, reviews

In [140]:
get_reviews('https://www.healthgrades.com/physician/dr-craig-ehrensing-yxw7q')

('Male',
 ['My visit was very pleasant, Dr Ehrensing is an amazing guy, he sat and listened, made me feel like family, very caring, that’s not easy to find these days, I highly recommend him, keep up the great work you do Doc, Gods Blessings always',
  'Very engaging and informative visit that addresses all questions.',
  "He is the BEST of the Best! I'm so glad I found him years ago when I was looking for a PCD. Thank You for all you do DR. E\n\nThe only reason I'm giving a 2 star for urgent appointment is because he he liked so much that everybody want to go to him.",
  'I think he is an excellent diagnostician and has a great personality besides. What else could you want in a doctor.',
  'I would not recommend Dr. Ehrensing to family and friends.  We have tried to get in touch with him or his nurse and did not get a return phone call in 5 days.  Have been very unhappy with the service the last few times we have called.'])

In [141]:
url2 = 'https://www.healthgrades.com/physician/dr-paula-hall-ypy2q'
get_reviews(url2)

('Female',
 ["Dr. Hall is an excellent physician with a wonderful bedside manner! She made me feel very at ease as to what was going to happen during my procedure and afterward. She goes over the results so you understand what has taken place. \nThe staff is also great in regards to the post-op instructions.   Dr. Hall has done my routine colonoscopy's for over 10 years- I would highly recommend her to others! ",
  'Dr. Hall is an excellent physician with a wonderful bedside manner! She made me feel very at ease as to what was going to happen during my procedure and afterward. Her staff was also great in regard to the pre-op instructions. So glad that I chose Dr. Hall for my routine colonoscopy- I will recommend her highly to others! ',
  'Dr. Hall is an outstanding physician. She has excellent bedside manner, takes time to thoroughly explain the procedure, what to expect, describes what she will do if she encounters any abnormalities and answers any questions you have.  I highly recom

In [137]:
link = 'https://www.healthgrades.com/find-a-doctor/a-z/a-1'
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
response = requests.get(link, headers=headers)
soup = BeautifulSoup(response.content)
soup.prettify()
lnks = soup.findAll("a")
docs = []
for i in range(len(lnks)):
    trial = re.findall(r"<a href=\"/physician+/+", str(lnks[i]))
    if trial == ['<a href="/physician/']:
        #print(i)
        #print(lnks[i])
        #print(re.findall(r'</?a(|\s+[^>]+)>',str(lnks[i])))
        cut = re.findall(r'</?a(|\s+[^>]+)>',str(lnks[i]))
        #print(cut[0][6:])
        #print(re.findall(r"<a href=(\"[a-z]+)", str(lnks[i])))
        docs.append(cut[0][7:-1])
docs

['/physician/dr-justin-a-de-boer-cjybz',
 '/physician/dr-ruben-a-6jx16p1158',
 '/physician/dr-enenge-abodjedi-25pd2',
 '/physician/dr-yousif-a-rahim-35982',
 '/physician/dr-melanie-aaberg-3vtmd',
 '/physician/dr-randal-aaberg-y5snb',
 '/physician/dr-thomas-aaberg-ynfm8',
 '/physician/dr-thomas-aaberg-xxkf9',
 '/physician/dr-afeera-aabida-xr46l',
 '/physician/dr-meghan-aabo-xylnfmj',
 '/physician/dr-stella-aaboe-xj54y',
 '/physician/dr-aazy-aaby-33pd2',
 '/physician/dr-douglas-aach-2r68l',
 '/physician/dr-james-aachen-winans-2s2r5',
 '/physician/dr-venkat-aachi-yhmh5',
 '/physician/dr-venkat-aachi-xymnppf',
 '/physician/dr-kirk-aadalen-y84qp',
 '/physician/dr-richard-aadalen-ybnh3',
 '/physician/dr-abdul-aadam-y3b4f',
 '/physician/dr-ahmed-aadil-xylh45p',
 '/physician/dr-timothy-aadland-25tsl',
 '/physician/dr-gregory-aaen-3fqqf',
 '/physician/dr-nancy-aaen-ydrx4',
 '/physician/dr-victor-aaen-yd7t5',
 '/physician/dr-bradley-aafedt-2tjgl',
 '/physician/dr-osama-aaflaq-3g3lk',
 '/physicia

In [144]:

revs = {}
for doc in docs[1:]:
    link = "https://www.healthgrades.com" + doc
    (gender, commentList) = get_reviews(link)
    for comment in commentList:
        revs[comment] = gender
revs


{'Exelent he really care about you, he is very professional,  and , compassionate.  Thanks Dr A ‘ dbojeri.': 'Male',
 'Dr. A-Rahim was very knowledgeable about my long term condition and spent a great deal of time explaining my treatment plan to me and my husband.': 'Male',
 'Horrible physician treats you like a kid. Pts c/o in er is never addressed by this md , she dances around anything you come in for. You will be told call your internal medicine dr because nothing is wrong, horrible bedside manners and argues with basic pt care ': 'Female',
 'I am privileged to have Dr Aaberg in charge of my eye care.': 'Male',
 'I have been seeing Dr. Aaberg for a number of years now.  He excels as a physician and diagnostician.  He is conservative in his approach, which I value and truly appreciate.  \n\nHe has always taken as much time as I needed to explain complications and procedures.  He truly is a compassionate partner with you in the care of your precious eyesight and is outstanding in his

# Determining the Frequency of Gendered Language

### References
Scikit-learn: Machine Learning in Python, Pedregosa et al., JMLR 12, pp. 2825-2830, 2011.

In [145]:
from sklearn.feature_extraction.text import CountVectorizer

#get list of 'important' words. We can use this list to identitfy frequencies of gendered words 
vect = CountVectorizer()
vect.fit(revs)
vect.get_feature_names()


['00',
 '10',
 '100',
 '101',
 '10am',
 '10mg',
 '11',
 '12',
 '1314',
 '15',
 '16th',
 '17',
 '18',
 '19',
 '1986',
 '1997',
 '1999',
 '1st',
 '20',
 '2008',
 '2012',
 '2013',
 '2014',
 '2016',
 '2018',
 '2019',
 '2020',
 '20mins',
 '21',
 '21st',
 '22',
 '24',
 '25',
 '250',
 '27',
 '29',
 '2nd',
 '30',
 '32',
 '35',
 '3rd',
 '40',
 '41',
 '438',
 '45',
 '47years',
 '48',
 '50',
 '530pm',
 '540',
 '55',
 '70',
 '73',
 '7am',
 '80',
 '85',
 '88',
 '8pm',
 '90',
 '9th',
 'aaaker',
 'aaberg',
 'aaby',
 'aach',
 'aachi',
 'aacker',
 'aadalen',
 'aadam',
 'aadamm',
 'aadom',
 'aaen',
 'aafreen',
 'aagaard',
 'aagard',
 'aagensen',
 'aaker',
 'aalami',
 'aalbers',
 'aalderink',
 'aaltonen',
 'aalund',
 'aamer',
 'aamir',
 'aamlid',
 'aamodt',
 'aamoth',
 'aanderud',
 'aanestad',
 'aanosen',
 'aarabi',
 'aaranson',
 'aardema',
 'aaron',
 'aarons',
 'aaronson',
 'ab',
 'abated',
 'abby',
 'abdominal',
 'abdul',
 'abilities',
 'ability',
 'able',
 'abnormal',
 'about',
 'above',
 'absolute',
